# Forecasting Baselines - Evaluating the results

## Setup

In [ ]:
from typing import Dict, List, Optional

import pickle
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from rich.progress import track
from rich.console import Console
from darts import TimeSeries
from pyro.ops.stats import crps_empirical
import torch
from torchmetrics import functional as F
from einops import rearrange

from proprietary_data import (
    companyid_to_name,
    ALL_FEATURE_NAMES,
    KEY_FEATURE_NAMES,
    CompanyFundamentalsKind,
    get_data_transform,
    get_adjusted_inverse_transform,
)
from proprietary_data.darts import (
    load_company_fundamentals_as_darts_time_series,
    load_company_fundamental_human_analysts_as_darts_time_series,
)
from forecasting_cfs.eval_model import ForecastingResult

In [ ]:
def crps_empirical_numpy(pred: np.ndarray, truth: np.ndarray) -> np.ndarray:
    return crps_empirical(torch.tensor(pred), torch.tensor(truth)).numpy()

In [ ]:
OUT_PATH = (
    Path("forecast_baselines")
    / "statics_True-revin_True-stride1_lb12-expanding-uq_True"
)
assert OUT_PATH.exists(), f"Path {OUT_PATH.absolute()} does not exist"

In [ ]:
palette: str = "muted"
sns.set_style("whitegrid")

In [ ]:
PLOT_FOR_ALL_MODELS = False

In [ ]:
EXCLUDE = {"ARIMA(4,1,4)", "VARIMA(4,0,4)", "VARIMA(4,1,4)"}  # Too unstable

## For a single fold

In [ ]:
SINGLE_OUT_PATH = OUT_PATH / "0" / "result_data"

### Load the data and prepare it

In [ ]:
df_single_fold: pd.DataFrame = pd.read_pickle(SINGLE_OUT_PATH / "results.pkl")
df_single_fold

In [ ]:
UNCEARTAINTY = df_single_fold.empty
UNCEARTAINTY

In [ ]:
df_single_fold.dropna(inplace=True)

In [ ]:
df_single_fold["metric"][
    df_single_fold["metric"].str.contains("(avg)", regex=False)
].unique().tolist()

In [ ]:
with open(SINGLE_OUT_PATH / "example_predictions.pkl", "rb") as f:
    example_predictions: Dict[str, List[ForecastingResult]] = pickle.load(f)

one_example_prediction = next(iter(example_predictions.values()))
len(example_predictions), len(one_example_prediction)

In [ ]:
N_SAMPLES = one_example_prediction[0].ts_forecast.n_samples if UNCEARTAINTY else None
N_SAMPLES

In [ ]:
HORIZON_LOOKBACK = one_example_prediction[0].ts_past.n_timesteps
HORIZON_LOOKBACK

In [ ]:
HORIZON_FORECAST = one_example_prediction[0].ts_forecast.n_timesteps
HORIZON_FORECAST

In [ ]:
NUM_FEATURES = one_example_prediction[0].ts_ground_truth.n_components
NUM_FEATURES

In [ ]:
match NUM_FEATURES:
    case 19:
        FEATURE_NAMES = ALL_FEATURE_NAMES
    case 5:
        FEATURE_NAMES = KEY_FEATURE_NAMES
    case _:
        raise ValueError(f"Unexpected number of features: {NUM_FEATURES}")

FEATURE_NAMES

In [ ]:
company_id_to_name = companyid_to_name(subset=False, min_length="max")
inverse_transform = get_adjusted_inverse_transform(FEATURE_NAMES)
inverse_transform_human_analyst = get_adjusted_inverse_transform(
    ["Revenue F12M Analyst Estimate"]
)

In [ ]:
def melt_avg(df: pd.DataFrame) -> pd.DataFrame:
    df_avg = df[df["metric"].str.endswith("(avg)")]
    # assert df_avg.index.is_unique
    return df_avg.astype({"value": float})


melt_avg(df_single_fold).head()

In [ ]:
def melt_avg_for_human(df: pd.DataFrame) -> pd.DataFrame:
    df_avg = df[df["metric"].str.endswith("(avg as human eval)")]
    # assert df_avg.index.is_unique
    return df_avg.astype({"value": float})


melt_avg(df_single_fold).head()

In [ ]:
def melt_lookahead(df: pd.DataFrame) -> pd.DataFrame:
    df_lh_molten = df[df["metric"].str.endswith("(avg per lookahead)")]
    df_lh_molten = df_lh_molten.explode("value", ignore_index=False)
    df_lh_molten["look-ahead"] = (np.arange(len(df_lh_molten)) % HORIZON_FORECAST) + 1
    # assert df_lh_molten.index.is_unique
    return df_lh_molten.astype({"value": float})


melt_lookahead(df_single_fold).head()

In [ ]:
def melt_feature(df: pd.DataFrame) -> pd.DataFrame:
    df_feat_molten = df[df["metric"].str.endswith("(avg per feature)")]
    df_feat_molten = df_feat_molten.explode("value", ignore_index=False)
    df_feat_molten["feature"] = np.arange(len(df_feat_molten)) % NUM_FEATURES
    # assert df_feat_molten.index.is_unique
    return df_feat_molten.astype({"value": float})


melt_feature(df_single_fold).head()

### Plotting

#### Show aggregated prediction errors and error over look-ahead horizon

In [ ]:
def plot_comparison(
    df: pd.DataFrame,
    save_dir: Path,
    postfix: str = "",
    show_metrics: List[str] = ["MAE", "MAPE", "MSE", "R2", "RMSE", "RSE", "SMAPE"],
    # show_metrics: List[str] = ["nCRPS"],
    exclude: set[str] = EXCLUDE,
) -> None:
    df_single = melt_avg(df)
    df_single = df_single[
        df_single["metric"].str.split(" ", n=2, expand=True)[0].isin(show_metrics)
    ]
    df_single.reset_index(inplace=True)
    df_single = df_single[~df_single["model"].isin(exclude)]
    df_single.sort_values(by=["model"], inplace=True)

    df_lh_molten = melt_lookahead(df)
    df_single.sort_values(by=["model"], inplace=True)
    df_lh_molten = df_lh_molten[
        df_lh_molten["metric"].str.split(" ", n=2, expand=True)[0].isin(show_metrics)
    ]
    # make it categorical:
    df_lh_molten["look-ahead"] = df_lh_molten["look-ahead"].astype("str")
    df_lh_molten.reset_index(inplace=True)
    df_lh_molten = df_lh_molten[~df_lh_molten["model"].isin(exclude)]
    df_lh_molten.sort_values(by=["model", "look-ahead"], inplace=True)

    save_dir.mkdir(exist_ok=True, parents=True)

    plot: sns.FacetGrid = sns.catplot(
        data=df_single.reset_index(),
        x="model",
        y="value",
        hue="model",
        col="metric",
        col_wrap=1,
        kind="bar",
        palette=palette,
        sharex=True,
        sharey=False,
        legend=False,
        aspect=4,
    )
    plot.set_xticklabels(rotation=45, horizontalalignment="right")
    # plt.subplots_adjust(hspace=0.25)
    plt.savefig(save_dir / f"comparison_avg{postfix}.pdf", bbox_inches="tight")
    plt.show()
    plt.close()

    plot: sns.FacetGrid = sns.relplot(
        data=df_lh_molten.dropna(),
        x="look-ahead",
        y="value",
        hue="model",
        style="model",
        col="metric",
        kind="line",
        palette=palette,
        facet_kws={"sharey": False},
    )
    plt.savefig(save_dir / f"comparison_lookahead{postfix}.pdf")
    plt.show()
    plt.close()


if not UNCEARTAINTY:
    plot_comparison(df_single_fold, save_dir=SINGLE_OUT_PATH.parent / "plots")

#### Show error per feature

In [ ]:
def plot_per_feature(
    df: pd.DataFrame,
    save_dir: Path,
    selected_metric: str = "SMAPE (avg per feature)",
    title: str = "SMAPE per model and feature",
):
    df_feat_molten_selected = melt_feature(df[df["metric"] == selected_metric])
    heatmap_data = (
        df_feat_molten_selected.reset_index()
        .pivot(columns="feature", index="model", values="value")
        .sort_values(by=["feature"], axis="columns")
    )
    heatmap_data.rename(columns=FEATURE_NAMES.__getitem__, inplace=True)

    df_feat_molten_selected["feature"] = df_feat_molten_selected["feature"].apply(
        FEATURE_NAMES.__getitem__
    )

    with sns.axes_style("white"):
        g = sns.JointGrid(
            data=df_feat_molten_selected,
            x="feature",
            y="model",
            hue="value",
            marginal_ticks=True,
            # ratio=10,
        )

        # Create an inset legend for the histogram colorbar
        cax = g.figure.add_axes([0.9, 0.85, 0.015, 0.12])

        # Add the joint and marginal histogram plots
        h = sns.heatmap(
            data=heatmap_data,
            ax=g.ax_joint,
            cbar_ax=cax,
            annot=True,
            fmt=".3f",
            annot_kws=dict(fontsize="xx-small"),
        )
        cmap = cax.collections[1].get_cmap()
        cmap_norm = cax.collections[1].norm
        h.set_xticklabels(h.get_xticklabels(), rotation=45, horizontalalignment="right")
        # b = sns.barplot(
        #     data=df_feat_molten_selected,
        #     ax=g.ax_marg_x,
        #     x="feature",
        #     y="value",
        #     orient="v",
        #     color="skyblue",
        # )
        # sns.barplot(
        #     data=df_feat_molten_selected.reset_index(),
        #     ax=g.ax_marg_y,
        #     y="model",
        #     x="value",
        #     orient="h",
        #     color="skyblue",
        # )
        bar_data = (
            df_feat_molten_selected.groupby(["feature"])["value"].mean().to_numpy()
        )
        g.ax_marg_x.bar(
            g.ax_joint.get_xticks(),
            bar_data,
            align="center",
            color=cmap(cmap_norm(bar_data)),
        )
        barh_data = (
            df_feat_molten_selected.groupby(["model"])["value"].mean().to_numpy()
        )
        g.ax_marg_y.barh(
            g.ax_joint.get_yticks(),
            barh_data,
            align="center",
            color=cmap(cmap_norm(barh_data)),
        )

        plt.subplots_adjust(top=0.95)
        plt.suptitle(title)

    plt.savefig(save_dir / "error_per_feature.pdf", bbox_inches="tight")
    plt.show()
    plt.close()


if not UNCEARTAINTY:
    plot_per_feature(df_single_fold, save_dir=SINGLE_OUT_PATH.parent / Path("plots"))

#### Show example predictions

In [ ]:
def prediction_result_to_df(
    results: list[ForecastingResult], first_n_companies: int = 15
):
    df = pd.DataFrame(
        [
            {
                "companyid": int(result.meta_data["companyid"]),
                "companyname": company_id_to_name[int(result.meta_data["companyid"])],
                # "GICS_sector": int(result.meta_data["GICS_sector"]),
                # "GICS_industry_group": int(result.meta_data["GICS_industry_group"]),
                "metadata": repr(result.meta_data),
                "time": time,
                "label": mode,
                "variable": component_name,
                "value_transformed": value_transformed,
                "value": value,
            }
            for result in results[:first_n_companies]
            if result.ts_forecast is not None
            for ts, mode in zip(
                (
                    result.ts_past,
                    # This connects the past and future/predicted time series with a line
                    result.ts_ground_truth.prepend(result.ts_past[-1:]),
                    result.ts_forecast.prepend_values(
                        result.ts_past[-1:].values()[..., None][
                            ..., [0] * result.ts_forecast.n_samples
                        ]
                    ),
                ),
                ["past", "future", "forecast"],
            )
            for row_transformed, row, component_name in zip(
                ts.all_values().T,
                inverse_transform(ts.all_values()).T,
                ts.components,
            )
            for time, value_transformed, value in zip(
                ts.time_index, row_transformed, row
            )
        ]
    )
    df = df.set_index("companyid")
    return df


# prediction_result_to_df(example_predictions["RNN (GRU)"])

In [ ]:
# is_first = True
# for model, results in track(
#     list(example_predictions.items()), description="Plotting per model"
# ):
#     data = prediction_result_to_df(results)
#     for variable in ("value", "value_transformed"):
#         sns.relplot(
#             data=data,
#             col="variable",
#             hue="companyname",
#             col_wrap=4,
#             x="time",
#             y=variable,
#             style="label",
#             kind="line",
#             aspect=1.5,
#             facet_kws={"sharey": False, "sharex": True},
#         )
#         directory = SINGLE_OUT_PATH.parent / "plots" / "concrete_examples"
#         directory.mkdir(exist_ok=True, parents=True)
#         plt.savefig(directory / f"{model}_{variable}.pdf")
#         if is_first:
#             plt.show()
#             is_first = False

#         plt.close()

#     if not PLOT_FOR_ALL_MODELS:
#         print("Limiting to the first model to be faster")
#         break

## Overview over multiple folds

In [ ]:
# Use the exact same setup as when training the models

darts_ts = load_company_fundamentals_as_darts_time_series(
    kind=CompanyFundamentalsKind.Normalized,
    subset=False,
    target_columns=FEATURE_NAMES,
    covariate_columns=[item for item in ALL_FEATURE_NAMES if item not in FEATURE_NAMES],
    # only use complete sequences for simplicity when comparing to other models:
    min_length="max",
)

darts_ts_human_analyst = load_company_fundamental_human_analysts_as_darts_time_series(
    kind=CompanyFundamentalsKind.Normalized,
    subset=False,
    # only use complete sequences for simplicity when comparing to other models:
    min_length="max",
)

### Data preparation

In [ ]:
# list of different timestamps with each a mapping of: model -> companyid -> timeseries
forecasts: list[dict[str, dict[int, TimeSeries]]] = []

for split_directory in track(
    list(
        sorted(
            filter(
                lambda path: path.is_dir() and not path.name == "plots",
                OUT_PATH.iterdir(),
            ),
            key=lambda path: int(path.name),
        )
    ),
    description="Loading predictions",
):
    with open(split_directory / "result_data" / "example_predictions.pkl", "rb") as f:
        forecasts.append(pickle.load(f))


len(forecasts)

In [ ]:
try:
    ORDER_MODELS = list(sorted(forecasts[0]))
    for model in EXCLUDE:
        if model in ORDER_MODELS:
            ORDER_MODELS.remove(model)

    ORDER_COMPANIES = list(
        sorted(
            set(
                forecast_result.meta_data["companyid"]
                for forecast_result in forecasts[0][ORDER_MODELS[0]]
            )
        )
    )
except NameError:
    raise RuntimeError("the forecasts_slim is not trustworthy")

    ORDER_MODELS = list(sorted(forecasts_slim[0]))
    for model in EXCLUDE:
        if model in ORDER_MODELS:
            ORDER_MODELS.remove(model)

    ORDER_COMPANIES = list(sorted(set(forecasts_slim[0][ORDER_MODELS[0]].keys())))

In [ ]:
# An example time series:
# forecasts[0][ORDER_MODELS[0]][18671]

In [ ]:
models_per_window = {
    forecast_id: list(sorted(forecast.keys()))
    for forecast_id, forecast in enumerate(forecasts)
}
df_models_per_window = (
    pd.DataFrame(models_per_window.items(), columns=["window", "models"])
    .explode("models")
    .groupby("models")
    .count()
    .rename(columns={"window": "present in # windows"})
)
df_models_per_window

In [ ]:
split_index_to_start_time = {
    split_index: (
        forecast[ORDER_MODELS[0]][0]
        .ts_forecast.start_time()
        .strftime("%Y %m")
        .replace(" 01", " Q1")
        .replace(" 04", " Q2")
        .replace(" 07", " Q3")
        .replace(" 10", " Q4")
    )
    for split_index, forecast in enumerate(forecasts)
}
split_index_to_start_time  # Also defined manually down for the plots

In [ ]:
ground_truth_per_company = {
    company_id: darts_ts.where_meta_data_matches(
        key="companyid", value=company_id
    ).targets[0]
    for company_id in company_id_to_name
}
len(ground_truth_per_company)

In [ ]:
huamn_analyst_per_company = {
    company_id: darts_ts_human_analyst.where_meta_data_matches(
        key="companyid", value=company_id
    ).targets[0]
    for company_id in company_id_to_name
}
len(huamn_analyst_per_company) == len(ground_truth_per_company)

In [ ]:
_ground_truth_data_processed = pd.DataFrame(
    [
        {
            "split": "true",
            "model": None,
            "company_id": company_id,
            "time": time,
            "label": "true",
            "variable": component_name,
            "EUR transformed": value_transformed,
            # "EUR": value,
        }
        for company_id, ground_truth in ground_truth_per_company.items()
        for row_transformed, row, component_name in zip(
            ground_truth.all_values().squeeze(-1).T,
            ground_truth.all_values().squeeze(-1).T,
            # inverse_transform(ground_truth.all_values().squeeze(-1)).T,  # TODO
            ground_truth.components,
        )
        for time, value_transformed, value in zip(
            ground_truth.time_index, row_transformed, row
        )
    ]
)
_ground_truth_data_processed.to_parquet(
    OUT_PATH / "ground_truth_data_processed.parquet"
)
_ground_truth_data_processed.info()

In [ ]:
_ground_truth_data_processed = pd.read_parquet(
    OUT_PATH / "ground_truth_data_processed.parquet"
)

In [ ]:
_human_analyst_data_processed = pd.DataFrame(
    [
        {
            "split": "human analyst",
            "model": "human analyst",
            "company_id": company_id,
            "time": time,
            "label": "human analyst",
            "variable": component_name,
            "EUR transformed": value_transformed,
            "EUR": value,
        }
        for company_id, ground_truth in huamn_analyst_per_company.items()
        for row_transformed, row, component_name in zip(
            ground_truth.all_values().squeeze(-1).T,
            ground_truth.all_values().squeeze(-1).T,
            # inverse_transform_human_analyst(ground_truth.all_values().squeeze(-1)).T,  # TODO
            ground_truth.components,
        )
        for time, value_transformed, value in zip(
            ground_truth.time_index, row_transformed, row
        )
    ]
)
_human_analyst_data_processed.to_parquet(
    OUT_PATH / "human_analyst_data_processed.parquet"
)
_human_analyst_data_processed.info()

In [ ]:
_human_analyst_data_processed = pd.read_parquet(
    OUT_PATH / "human_analyst_data_processed.parquet"
)

In [ ]:
ts = forecasts[0][ORDER_MODELS[0]][2].ts_forecast
ts

In [ ]:
data_transform = get_data_transform(subset=False, min_length="max")

In [ ]:
def transform_back_to_original(
    time_series: TimeSeries, company_id: int, normalize_date=None
) -> TimeSeries:
    as_array = np.zeros(
        (
            time_series.n_timesteps,
            len(data_transform._feature_names),
            time_series.n_samples,
        )
    )
    indices = [
        data_transform._feature_names.index(feat) for feat in time_series.components
    ]
    as_array[:, indices, :] = time_series.all_values()
    inverted = data_transform.standardizer.inverse_transform(
        rearrange(as_array, "time component sample -> (time sample) component")
    )
    inverted = rearrange(
        # Restrict to the components of the time series
        inverted[:, indices],
        "(time sample) component -> time component sample",
        time=time_series.n_timesteps,
    )

    normalizer = data_transform._source_data[
        data_transform._source_data["companyid"] == company_id
    ]
    if normalize_date is None:
        normalizer = normalizer[normalizer["aca_quarter"] < ts.time_index.min()]
        normalizer = normalizer[
            normalizer["aca_quarter"] == normalizer["aca_quarter"].max()
        ]
    else:
        normalizer = normalizer[normalizer["aca_quarter"] == normalize_date]
    assert len(normalizer) == 1, normalizer

    for col, col_normalizer in data_transform._normalizer_mapping.items():
        if col in time_series.components:
            index = time_series.components.get_loc(col)
            inverted[:, index, :] = inverted[:, index, :] * (
                normalizer[col_normalizer].item() + data_transform.eps
            )

    return inverted


transform_back_to_original(ts, company_id=18527).shape

In [ ]:
def make_df_for_company_results(
    forecasts, model: str, include_companies: Optional[List[int]] = None
) -> pd.DataFrame:
    include_company_indices = [ORDER_COMPANIES.index(c) for c in include_companies]
    df_specific = pd.DataFrame(
        [
            {
                "split": f"forecast #{split}",
                "model": model,
                "company_id": ORDER_COMPANIES[company_index],
                "time": time,
                "label": label,
                "variable": component_name,
                "EUR transformed": value_transformed,
                "EUR": value,
                "sample_idx": sample_idx,
            }
            for split, ts_in_split in enumerate(
                track(
                    forecasts,
                    description=f"Computing data for model '{model}' for various splits",
                    disable=True,
                )
            )
            for company_index, result in enumerate(ts_in_split[model])
            if result is not None
            and (include_companies is None or company_index in include_company_indices)
            for ts_unconnected in [result.ts_forecast]
            # for ground_truth in [ground_truth_per_company[company_id]]
            for ts in [
                # # This is a hacky way to connect the past and future/predicted time series with a line
                ts_unconnected.prepend_values(
                    # values has shape (time, component, sample),
                    result.ts_past.all_values()[-1:, :, [0] * ts_unconnected.n_samples]
                    # ground_truth[
                    #     ground_truth.get_index_at_point(
                    #         ground_truth._get_last_timestamp_before(
                    #             ts_unconnected.start_time()
                    #         )
                    #     )
                    #     - 1 : ground_truth.get_index_at_point(
                    #         ground_truth._get_last_timestamp_before(
                    #             ts_unconnected.start_time()
                    #         )
                    #     )
                    # ].values()[..., None][..., [0] * ts_unconnected.n_samples]
                )
            ]
            for ts, label in [
                (ts, "forecast"),
                # ts_ground_truth and ts_past only contain the testing split data, so not the full data!
                *(
                    # Add all the data from the last split, since it contains all of it
                    [(result.ts_past, "true"), (result.ts_ground_truth, "true")]
                    if split == 0
                    else [(result.ts_ground_truth, "true")]
                ),
            ]
            for sample_idx in range(ts.n_samples)  # Inefficient order, but works
            for row_transformed, row, component_name in zip(
                ts.values(sample=sample_idx).T,
                # ts.all_values()[..., sample_idx].T,
                transform_back_to_original(
                    ts, company_id=ORDER_COMPANIES[company_index]
                )[:, :, sample_idx].T,
                ts.components,
            )
            for time, value_transformed, value in zip(
                ts.time_index,
                row_transformed,
                row,
            )
        ]
    )
    return pd.concat(
        [
            df_specific
        ],  # _ground_truth_data_processed, _human_analyst_data_processed],  # TODO exclude irrelevant companies here
        ignore_index=True,
    )


# make_df_for_company_results(
#     forecasts, "Prophet", include_companies=list(company_id_to_name.keys())[:1]
# )

For the market evaluation, we need:
- company_id
- last_known_date
- feature
- forecasting_step (the zeroth quarter is the ground truth)
- value

In [ ]:
def make_df_for_export(forecasts, model: str) -> pd.DataFrame:
    return pd.DataFrame(
        [
            {
                "company_id": ORDER_COMPANIES[company_index],
                "last_known_time": last_known_time,
                "step": step,
                "variable": component_name,
                "EUR transformed": value_transformed,
                "EUR": value,
                "EUR transformed (std)": std_transformed,
                "EUR (std)": std,
            }
            for split, ts_in_split in enumerate(
                track(
                    forecasts  # disable=True,
                )
            )
            # if split in [34, 35, 36, 37, 38, 39, 40]  # TODO
            for company_index, result in enumerate(ts_in_split[model])
            if result is not None  # and company_index in [0, 1]  # TODO
            for ts_mean_forecast in [result.ts_forecast.mean()]
            for ts_std_forecast in [result.ts_forecast.std()]
            for normalize_date in [result.ts_past.end_time()]
            for fc_transformed, fc, fc_transformed_std, fc_std, past_transformed, past, gt_transformed, gt, component_name in zip(
                ts_mean_forecast.values().T,
                transform_back_to_original(
                    ts_mean_forecast,
                    company_id=ORDER_COMPANIES[company_index],
                    normalize_date=normalize_date,
                )[:, :, 0].T,
                ts_std_forecast.values().T,
                transform_back_to_original(
                    ts_std_forecast,
                    company_id=ORDER_COMPANIES[company_index],
                    normalize_date=normalize_date,
                )[:, :, 0].T,
                result.ts_past.values()[-1:, :].T,
                transform_back_to_original(
                    result.ts_past[-1:],
                    company_id=ORDER_COMPANIES[company_index],
                    normalize_date=normalize_date,
                )[:, :, 0].T,
                result.ts_ground_truth.values()[:, :].T,
                transform_back_to_original(
                    result.ts_ground_truth,
                    company_id=ORDER_COMPANIES[company_index],
                    normalize_date=normalize_date,
                )[:, :, 0].T,
                ts.components,
            )
            for step, value_transformed, value, std_transformed, std, last_known_time in [
                (0, past_transformed.item(), past.item(), None, None, normalize_date),
                *[
                    (
                        step + 1,
                        fc_transformed[step].item(),
                        fc[step].item(),
                        fc_transformed_std[step].item(),
                        fc_std[step].item(),
                        normalize_date,
                    )
                    for step in range(result.ts_forecast.n_timesteps)
                ],
                *(
                    []
                    if split < len(forecasts) - 1
                    else [
                        (
                            0,
                            gt_transformed[step].item(),
                            gt[step].item(),
                            None,
                            None,
                            result.ts_ground_truth.time_index[step],
                        )
                        for step in range(result.ts_ground_truth.n_timesteps)
                    ]
                ),
            ]
        ]
    )


# model = "RNN (GRU)"
# export_df = make_df_for_export(forecasts, model)
# export_df.to_parquet(OUT_PATH / f"for_market_eval_{model}.parquet")
# export_df

In [ ]:
# export_df["last_known_time"].value_counts()

In [ ]:
# export_df[export_df["last_known_time"] == "2022-07-01"]

In [ ]:
# !ls -lsh $OUT_PATH

In [ ]:
# export_df["last_known_time"].max()

In [ ]:
# len(export_df["company_id"].unique())

In [ ]:
# len(export_df["last_known_time"].unique())

In [ ]:
# export_df["step"].value_counts()

In [ ]:
# Turn it into an array of shape (n_splits, n_models, n_companies, n_timesteps, n_features, n_samples)
forecasts_array = np.full(
    (
        len(forecasts),
        len(ORDER_MODELS),
        len(ORDER_COMPANIES),
        HORIZON_FORECAST,
        NUM_FEATURES,
        N_SAMPLES or 1,
    ),
    # Make sure to use NaNs to indicate missing values in case we have a bug
    fill_value=np.nan,
    dtype=np.float64,
)
ground_truth_array = np.full(
    (
        len(forecasts),
        len(ORDER_MODELS),
        len(ORDER_COMPANIES),
        HORIZON_FORECAST,
        NUM_FEATURES,
        # No samples, only one dimension
    ),
    fill_value=np.nan,
    dtype=np.float64,
)

for split_num, split in enumerate(forecasts):
    for model_num, model in enumerate(ORDER_MODELS):
        for entry in split[model]:
            company_num = ORDER_COMPANIES.index(entry.meta_data["companyid"])

            ground_truth_array[split_num, model_num, company_num, ...] = (
                entry.ts_ground_truth.values()
            )

            fc = entry.ts_forecast
            if fc is not None:
                forecasts_array[split_num, model_num, company_num, ...] = (
                    fc.all_values()
                )

                if np.isnan(
                    forecasts_array[split_num, model_num, company_num, ...]
                ).any():
                    print(
                        f"NaNs in forecast for model '{model}' and company '{entry.meta_data['companyid']}'"
                    )

forecasts_array.shape

In [ ]:
np.count_nonzero(np.isnan(forecasts_array)) / forecasts_array.size * 100

In [ ]:
np.count_nonzero(np.isnan(ground_truth_array)) / ground_truth_array.size * 100

In [ ]:
# Find the time and model where thes is NaN (first two dimensions)
np.unique(np.argwhere(np.isnan(forecasts_array))[:, :2], axis=0)

In [ ]:
# ORDER_MODELS[3]

In [ ]:
data_array_path = OUT_PATH / "data_array.pkl"
ground_truth_array_path = OUT_PATH / "ground_truth_array.pkl"

if data_array_path.exists() and ground_truth_array_path.exists():
    with open(data_array_path, "rb") as f:
        forecasts_array = pickle.load(f)
    with open(ground_truth_array_path, "rb") as f:
        ground_truth_array = pickle.load(f)
else:
    with open(data_array_path, "wb") as f:
        pickle.dump(forecasts_array, f)
    with open(ground_truth_array_path, "wb") as f:
        pickle.dump(ground_truth_array, f)

forecasts_array.shape

In [ ]:
# preds = forecasts_array.mean(axis=-1, keepdims=True)[..., 0]
# targets = ground_truth_array

In [ ]:
# targets = targets[np.isfinite(preds)].reshape((40, 18, 2527, 4, 5))
# preds = preds[np.isfinite(preds).reshape(preds.shape)]

In [ ]:
if N_SAMPLES is None:  # forecasts are deterministic
    preds = forecasts_array.squeeze(-1)
    targets = ground_truth_array

    fh_dim = -2
    assert targets.shape[fh_dim] == HORIZON_FORECAST

    def mae(pred: np.ndarray, truth: np.ndarray) -> np.ndarray:
        return np.abs(pred - truth)

    def mse(pred: np.ndarray, truth: np.ndarray) -> np.ndarray:
        return np.square(pred - truth)

    def rmse(pred: np.ndarray, truth: np.ndarray) -> np.ndarray:
        overall = mse(pred, truth)
        result = np.sqrt(overall.mean(fh_dim))  # mean over horizon
        return np.stack([result] * overall.shape[fh_dim], axis=fh_dim)

    def mape(
        pred: np.ndarray, truth: np.ndarray, epsilon: float = 1.17e-06
    ) -> np.ndarray:
        return np.abs(pred - truth) / np.maximum(np.abs(truth), epsilon)

    def rse(
        pred: np.ndarray, truth: np.ndarray, epsilon: float = 1.17e-06
    ) -> np.ndarray:
        return np.square(pred - truth) / (
            np.square(truth - np.expand_dims(truth.mean(fh_dim), fh_dim)) + epsilon
        )

    def smape(
        pred: np.ndarray, truth: np.ndarray, epsilon: float = 1.17e-06
    ) -> np.ndarray:
        return np.abs(pred - truth) / np.maximum(
            np.abs(pred), np.maximum(np.abs(truth), epsilon)
        )

    def r2(
        pred: np.ndarray, truth: np.ndarray, epsilon: float = 1.17e-06
    ) -> np.ndarray:
        # mean over horizon
        result = np.sum(np.square(pred - truth), axis=fh_dim) / (
            np.sum(
                np.square(truth - np.expand_dims(truth.mean(fh_dim), fh_dim)),
                axis=fh_dim,
            )
            + epsilon
        )
        return 1 - np.stack([result] * pred.shape[fh_dim], axis=fh_dim)

In [ ]:
# the metrics now all have shape (n_splits, n_models, n_companies, n_timesteps, n_features)

if N_SAMPLES is None:
    results = {
        "MAE": mae(preds, targets),
        "MSE": mse(preds, targets),
        "RMSE": rmse(preds, targets),
        "MAPE": mape(preds, targets),
        "RSE": rse(preds, targets),
        "SMAPE": smape(preds, targets),
        "R2": r2(preds, targets),
    }
else:
    negative_crps = crps_empirical_numpy(
        np.moveaxis(forecasts_array, -1, 0), ground_truth_array
    )
    results = {"nCRPS": negative_crps}

len(results)

In [ ]:
if "nCRPS" in results:
    df_ncrps = pd.DataFrame(
        [
            {
                "split": split_num,
                "model": model,
                "metric": "nCRPS (avg)",
                "value": split[model_num, :, :, :].mean(axis=(0, 1, 2)).tolist(),
            }
            for split_num, split in enumerate(negative_crps)
            for model_num, model in enumerate(ORDER_MODELS)
        ]
        + [
            {
                "split": split_num,
                "model": model,
                "metric": "nCRPS (avg per feature)",
                "value": split[model_num, :, :, :].mean(axis=(0, 1)).tolist(),
            }
            for split_num, split in enumerate(negative_crps)
            for model_num, model in enumerate(ORDER_MODELS)
        ]
        + [
            {
                "split": split_num,
                "model": model,
                "metric": "nCRPS (avg per lookahead)",
                "value": split[model_num, :, :, :].mean(axis=(0, 2)).tolist(),
            }
            for split_num, split in enumerate(negative_crps)
            for model_num, model in enumerate(ORDER_MODELS)
        ]
    )

    pd.to_pickle(df_ncrps, OUT_PATH / "ncrps_df.pkl")

    df_ncrps

In [ ]:
# Turn into a dataframe with columns split, model, metric, horizon_step, feature, value

df_proper = pd.DataFrame(
    [
        {
            "split": split_num,
            "model": model,
            "metric": name,
            "company_id": ORDER_COMPANIES[company_idx],
            "feature": FEATURE_NAMES[feature_idx],
            "horizon_step": horizon_step,
            "value": split[model_num, company_idx, horizon_step, feature_idx].item(),
        }
        for name, results_array in track(results.items())
        for split_num, split in enumerate(results_array)
        for model_num, model in enumerate(ORDER_MODELS)
        for company_idx in range(len(ORDER_COMPANIES))
        for horizon_step in range(HORIZON_FORECAST)
        for feature_idx in range(NUM_FEATURES)
    ]
)

df_proper.to_parquet(OUT_PATH / "df_proper.parquet")

df_proper

In [ ]:
!ls -sh {OUT_PATH}

### Forecast per company

In [ ]:
sns.set_context("talk")

In [ ]:
def make_company_plot(df: pd.DataFrame) -> None:
    # company_id = df["company_id"].iloc[0]
    # company_name = company_id_to_name[company_id]
    grid = sns.relplot(
        data=df.rename(columns={"label": "Target Variates"}).replace(
            {"Target Variates": {"forecast": "Forecast", "true": "Ground Truth"}}
        ),
        col="variable",
        hue="Target Variates",
        col_wrap=3,
        x="time",
        y="EUR",
        # style="split",  # Too much information to be useful
        # style="Target Variates",  # Double legend to make it more obvious
        kind="line",
        aspect=1.5,
        facet_kws={"sharey": False, "sharex": True},
        errorbar=("pi", 68),
        palette="colorblind",
    )
    grid.figure.subplots_adjust(top=0.9)
    grid.set_titles("{col_name}")
    grid.set(xlim=(df["time"].min(), df["time"].max()))
    grid.set(xlabel="")

    # grid.figure.suptitle(f"{model} predicting {company_name}")

    # Only have a legend about the label, not the split
    # handles, labels = grid.axes[0].get_legend_handles_labels()
    # grid.legend.remove()
    # grid.axes[0].legend(handles[:2], labels[:2], title="Label")
    sns.move_legend(grid, "upper center", bbox_to_anchor=(0.775, 0.35))

    sns.despine(bottom=True)

    return grid

In [ ]:
PLOT_FOR_ALL_MODELS = False

is_first = True
console = Console()
for model in track(
    list(forecasts[0].keys()),
    description="Plotting for all models",
    console=console,
):
    break

    model = "RNN (GRU)"

    if (
        df_models_per_window.loc[model, "present in # windows"]
        < df_models_per_window["present in # windows"].max()
    ):
        print(f"WARNING: Skipping {model} because it is not present in all windows")
        continue

    index = 50
    interesting_company_ids = list(company_id_to_name.keys())[index : index + 20]

    for company_id in track(
        interesting_company_ids,
        description="Plotting for each company",
        console=console,
        disable=True,
    ):
        company_name = company_id_to_name[company_id]
        data_company = make_df_for_company_results(
            forecasts, model, include_companies=[company_id]
        )
        grid = make_company_plot(data_company)
        grid.figure.show()

        directory = OUT_PATH / "plots" / "concrete_examples" / model
        directory.mkdir(exist_ok=True, parents=True)
        plt.savefig(directory / f"{company_id}-{company_name}.pdf")
        if is_first:
            plt.show()
            is_first = False

            # if not PLOT_FOR_ALL_MODELS:
            #     print("Limiting to the first company to be faster")
            #     break

        plt.close()

    if not PLOT_FOR_ALL_MODELS:
        print("Limiting to the first model to be faster")
        break

In [ ]:
OUT_PATH / "plots" / "concrete_examples"

#### Error aggregated over all splits

In [ ]:
def load_results(base_path: Path, models: set[str] | None = None) -> pd.DataFrame:
    df_all = pd.concat(
        {
            path.name: pd.read_pickle(
                base_path / path.name / "result_data" / "results.pkl"
            )
            for path in base_path.iterdir()
            if (path.is_dir() and path.name.isdigit())
        },
        names=["split"],  # For the new index
    )
    df_all.reset_index(inplace=True)

    ncrps_path = base_path / "ncrps_df.pkl"
    if ncrps_path.exists():
        df_ncrps = pd.read_pickle(ncrps_path)
        df_all = pd.concat([df_all, df_ncrps], ignore_index=True)

    df_all["split"] = df_all["split"].astype(int)
    df_all = df_all.sort_values(by=["split"])
    if models is not None:
        df_all = df_all[df_all["model"].isin(models)]
    else:
        df_all = df_all[~df_all["model"].isin(EXCLUDE)]
    return df_all


df_all = load_results(OUT_PATH)
df_all

In [ ]:
OUT_PATH_AGGREGATED = OUT_PATH / "plots" / "aggregated"
OUT_PATH_AGGREGATED.mkdir(exist_ok=True, parents=True)
str(OUT_PATH_AGGREGATED)

In [ ]:
# plot_comparison(df_all, save_dir=OUT_PATH_AGGREGATED)

In [ ]:
df_all_avg_over_splits = (
    melt_feature(df_all)
    .groupby(["model", "metric", "feature"])  # not by split
    .mean()
    .drop(columns=["split"])
    .reset_index()
)
df_all_avg_over_splits.head()

# plot_per_feature(df_all_avg_over_splits, save_dir=OUT_PATH_AGGREGATED)

#### Error over Time

In [ ]:
def plot_error_over_time(
    df: pd.DataFrame, save_dir: Path, postfix: str = "", show_metric: str = "nCRPS"
) -> None:
    df_lh_molten = melt_feature(df)
    df_lh_molten.sort_values(by=["split", "model"], inplace=True)
    df_lh_molten = df_lh_molten[
        df_lh_molten["metric"] == f"{show_metric} (avg per feature)"
    ]
    df_lh_molten.drop(columns=["metric"], inplace=True)
    df_lh_molten.rename(columns={"value": show_metric}, inplace=True)

    df_lh_molten["feature"] = df_lh_molten["feature"].apply(FEATURE_NAMES.__getitem__)

    df_lh_molten["split"] = df_lh_molten["split"].apply(
        split_index_to_start_time.__getitem__
    )
    df_lh_molten.rename(columns={"split": "forecast start"}, inplace=True)

    save_dir.mkdir(exist_ok=True, parents=True)

    plot: sns.FacetGrid = sns.relplot(
        data=df_lh_molten,
        x="forecast start",
        y=show_metric,
        hue="model",
        style="model",
        col="feature",
        col_wrap=3,
        kind="line",
        palette=palette,
        facet_kws={
            "sharex": False,
            "sharey": True,  # Also show which features generally have the highest error:
        },
    )
    xticks = sorted(list(split_index_to_start_time.values()))
    plot.set_xticklabels(xticks, step=2, rotation=25, horizontalalignment="right")
    plt.subplots_adjust(hspace=0.25)
    plt.savefig(save_dir / f"comparison_error_over_time_per_feature{postfix}.pdf")
    plt.show()
    plt.close()


plot_error_over_time(df_all, save_dir=OUT_PATH_AGGREGATED)

# Comparison across directories

In [ ]:
OUT_PATH_CROSS = OUT_PATH.parent
OUT_PATH_CROSS

In [ ]:
sns.set_context("talk")

In [ ]:
df_det = load_results(
    Path("forecast_baselines")
    / "statics_True-revin_True-stride1_lb12-expanding-uq_False"
)

In [ ]:
df_uq = load_results(
    Path("forecast_baselines")
    / "statics_True-revin_True-stride1_lb12-expanding-uq_True"
)

In [ ]:
df_uq = df_uq[df_uq["model"] != "AutoTheta"]  # too unstable

In [ ]:
df_mae_ncrps = df_det[df_det["metric"].str.contains("MAE")].copy()

df_mae_ncrps.set_index(["split", "model", "metric"], inplace=True)
df_uq["metric"] = df_uq["metric"].str.replace("nCRPS", "MAE", regex=True)
probabilisitc_models = df_uq["model"].unique().tolist()
df_uq.set_index(["split", "model", "metric"], inplace=True)
df_mae_ncrps.update(df_uq)
df_mae_ncrps.reset_index(inplace=True)

df_mae_ncrps["metric"] = df_mae_ncrps["metric"].str.replace(
    "MAE", "nCRPS/MAE", regex=True
)

df_mae_ncrps

In [ ]:
probabilisitc_models

In [ ]:
df_mae_ncrps["metric"].unique()

In [ ]:
df_cross = pd.concat([df_det, df_mae_ncrps], ignore_index=True)
df_cross

In [ ]:
df_cross.dropna().groupby(["model"])["value"].count()

In [ ]:
df_cross.rename(columns={"model": "Models"}, inplace=True)

order = [
    "Mean",
    "ARMean(1)",
    "ARMean(4)",
    "ARMA(1,1)",
    "ARMA(4,4)",
    "ARIMA(4,1,4)",
    "VARIMA(4,0,4)",
    "VARIMA(4,1,4)",
    "AutoARIMA",
    "AutoTheta",
    "Prophet",
    "Linear Reg.",
    "Random Forest",
    "DLinear",
    "NLinear",
    "RNN (LSTM)",
    "RNN (GRU)",
    "Block RNN (LSTM)",
    "Block RNN (GRU)",
    "TCN",
    "Transformer",
    "TFT",
    "N-BEATS",
    "N-HiTS",
    "TiDE",
    "xLSTM-Mixer",
    "Chronos-Bolt-Small",
    "Chronos-Bolt-Base",
]
MODEL_NOT_QUITE_SHORT_NAMES = {
    "Block RNN (GRU)": "Block GRU",
    "Block RNN (LSTM)": "Block LSTM",
    "RNN (GRU)": "GRU",
    "RNN (LSTM)": "LSTM",
    "Chronos-Bolt-Small": "Chronos Small",
    "Chronos-Bolt-Base": "Chronos Base",
}
for model in set(order):  # make a copy!
    if model not in df_cross["Models"].unique():
        order.remove(model)
df_cross["Models"] = df_cross["Models"].replace(MODEL_NOT_QUITE_SHORT_NAMES)
order = [MODEL_NOT_QUITE_SHORT_NAMES.get(model, model) for model in order]

df_cross["Models"] = pd.Categorical(df_cross["Models"], order)
df_cross.sort_values(by=["Models", "metric"], inplace=True)

df_cross["metric"] = (
    df_cross["metric"].str.replace("SMAPE", "sMAPE").str.replace("R2", "R²")
)

MODEL_SHORT_NAMES = {
    "ARMean(1)": "ARMean\n(1)",
    "ARMean(4)": "ARMean\n(4)",
    "ARMA(1,1)": "ARMA\n(1,1)",
    "ARMA(4,4)": "ARMA\n(4,4)",
    "ARIMA(4,1,4)": "ARIMA\n(4,1,4)",
    "VARIMA(4,0,4)": "VARIMA\n(4,0,4)",
    "VARIMA(4,1,4)": "VARIMA\n(4,1,4)",
    "Linear Reg.": "Linear\nReg.",
    "Random Forest": "Rand.\nForest",
    "AutoARIMA": "Auto\nARIMA",
    "AutoTheta": "Auto\nTheta",
    "Block GRU": "Block\nGRU",
    "Block LSTM": "Block\nLSTM",
    "Transformer": "Trans-\nformer",
    "xLSTM-Mixer": "xLSTM-\nMixer",
    "Chronos Small": "Chronos\nSmall",
    "Chronos Base": "Chronos\nBase",
}
probabilisitc_models = probabilisitc_models + [
    after
    for before, after in MODEL_NOT_QUITE_SHORT_NAMES.items()
    if before in probabilisitc_models
]
probabilisitc_models = probabilisitc_models + [
    after
    for before, after in MODEL_SHORT_NAMES.items()
    if before in probabilisitc_models
]
probabilisitc_models

In [ ]:
df_cross["Models"].value_counts()

In [ ]:
# See https://matplotlib.org/stable/users/explain/colors/colormaps.html#colormaps

from matplotlib.cm import tab20, tab20b
from matplotlib.colors import ListedColormap

custom_tab = ListedColormap(
    tab20.colors + tab20b.colors[8:10] + tab20b.colors[1:2] + tab20b.colors[16:18], name="custom_tab", N=25
)
sns.set_palette(sns.color_palette(custom_tab.colors))
custom_tab

In [ ]:
# show_metrics: List[str] = [
#     "MAE",
#     "MAPE",
#     "MSE",
#     "R²",
#     "RMSE",
#     "RSE",
#     "sMAPE",
#     "sMAPE",
#     "nCRPS/MAE",
# ]
show_metrics: List[str] = ["sMAPE", "MSE"]
# show_metrics: List[str] = ["nCRPS/MAE"]

df_single = melt_avg(df_cross)
df_single["metric"] = df_single["metric"].str.split(" ", n=2, expand=True)[0]

df_single.sort_values(by=["metric", "Models"], inplace=True)
df_single = df_single.reset_index()

df_single_plot = df_single[df_single["metric"].isin(show_metrics)].copy()
df_single_plot["Models"] = df_single_plot["Models"].replace(MODEL_SHORT_NAMES)

# add "*" to the model it is is in probabilisitc_models
if "nCRPS/MAE" in show_metrics:
    df_single_plot["Models"] = df_single_plot["Models"].apply(
        lambda x: f"{x}*" if x not in probabilisitc_models else x
    )

plot: sns.FacetGrid = sns.catplot(
    data=df_single_plot,
    x="Models",
    y="value",
    hue="Models",
    col="metric",
    col_wrap=1,
    kind="bar",
    # palette=custom_tab,
    sharex=True,
    sharey=False,
    legend=False,
    aspect=4.5,
)
plot.tick_params(labelsize="small")
plot.set_titles("{col_name}")
for ax in plot.axes.flat:
    title = ax.title.get_text()
    ax.set_ylabel(title)
    ax.set_title("")

    # Make vertically dashed lines to the right of the models
    at = [7, 9, 22]
    for i in at:
        ax.axvline(i + 0.5, color="black", linestyle="--", linewidth=1.0, alpha=0.5)

# Section labels between teh dashed lines (as titles on top)
labels = ["Local", "Global", "Deep Learning", "Pretrained"]
first_ax = plot.axes.flat[0]
for i, last_i, label in zip(
    at + [df_single_plot["Models"].unique().size - 1], [-1] + at, labels
):
    first_ax.text(
        i - (i - last_i) / 2 + 0.5,
        # 0.45,
        1.81,
        label,
        ha="center",
        va="center",
        alpha=0.75,
        fontsize="smaller",
    )

# first_ax.set_ylim(None, 0.3)

suffix = "nCRPS" if "nCRPS/MAE" in show_metrics else "det"
plt.savefig(OUT_PATH_CROSS / f"comparison_avg_paper_{suffix}.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
show_metrics: List[str] = ["sMAPE", "MSE", "nCRPS/MAE"]

df_lh_molten = melt_lookahead(df_cross)
df_lh_molten = df_lh_molten[
    df_lh_molten["metric"].str.split(" ", n=2, expand=True)[0].isin(show_metrics)
]
df_lh_molten["look-ahead"] = df_lh_molten["look-ahead"].astype("str")
df_lh_molten["metric"] = df_lh_molten["metric"].str.split(" ", n=2, expand=True)[0]
# Fix the order of the models
df_lh_molten["metric"] = pd.Categorical(df_lh_molten["metric"], show_metrics)

df_lh_molten.rename(columns={"look-ahead": "quarters look-ahead"}, inplace=True)

plot: sns.FacetGrid = sns.relplot(
    data=df_lh_molten.dropna(),
    x="quarters look-ahead",
    y="value",
    hue="Models",
    style="Models",
    col="metric",
    kind="line",
    # palette=custom_tab,
    facet_kws={"sharey": False},
)
plot.set_titles("{col_name}")
for ax in plot.axes.flat:
    title = ax.title.get_text()
    ax.set_ylabel(title)
    ax.set_title("")
sns.move_legend(plot, "center right", bbox_to_anchor=(1.2, 0.5), ncol=2)
plot.set(
    xlim=(
        df_lh_molten["quarters look-ahead"].min(),
        df_lh_molten["quarters look-ahead"].max(),
    )
)
# plot.set(ylim=(0, None))
plt.subplots_adjust(wspace=0.45)
# plot.tight_layout()  # Adjust the subplot layout
plt.savefig(OUT_PATH_CROSS / "comparison_lookahead_paper.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
split_index_to_start_time = {
    0: "2013 Q1",
    1: "2013 Q2",
    2: "2013 Q3",
    3: "2013 Q4",
    4: "2014 Q1",
    5: "2014 Q2",
    6: "2014 Q3",
    7: "2014 Q4",
    8: "2015 Q1",
    9: "2015 Q2",
    10: "2015 Q3",
    11: "2015 Q4",
    12: "2016 Q1",
    13: "2016 Q2",
    14: "2016 Q3",
    15: "2016 Q4",
    16: "2017 Q1",
    17: "2017 Q2",
    18: "2017 Q3",
    19: "2017 Q4",
    20: "2018 Q1",
    21: "2018 Q2",
    22: "2018 Q3",
    23: "2018 Q4",
    24: "2019 Q1",
    25: "2019 Q2",
    26: "2019 Q3",
    27: "2019 Q4",
    28: "2020 Q1",
    29: "2020 Q2",
    30: "2020 Q3",
    31: "2020 Q4",
    32: "2021 Q1",
    33: "2021 Q2",
    34: "2021 Q3",
    35: "2021 Q4",
    36: "2022 Q1",
    37: "2022 Q2",
    38: "2022 Q3",
    39: "2022 Q4",
}

In [ ]:
df_lh_molten = melt_feature(df_cross)

show_metric: str = "nCRPS/MAE"
# show_metric: str = "nCRPS"

df_lh_molten.sort_values(by=["split", "Models"], inplace=True)
df_lh_molten = df_lh_molten[
    df_lh_molten["metric"] == f"{show_metric} (avg per feature)"
]
df_lh_molten.drop(columns=["metric"], inplace=True)
df_lh_molten.rename(
    columns={"value": show_metric, "forecast start": "Forecast Start"},
    inplace=True,
)

df_lh_molten["feature"] = df_lh_molten["feature"].apply(FEATURE_NAMES.__getitem__)

df_lh_molten["Forecast Start"] = df_lh_molten["split"].apply(
    split_index_to_start_time.__getitem__
)

# df_lh_molten = df_lh_molten[df_lh_molten["Models"] != "Linear Reg."]

plot: sns.FacetGrid = sns.relplot(
    data=df_lh_molten,
    x="Forecast Start",
    y=show_metric,
    hue="Models",
    style="Models",
    col="feature",
    col_wrap=3,
    kind="line",
    # palette=custom_tab,
    facet_kws={
        "sharex": False,
        "sharey": False,
    },
    err_kws=dict(
        alpha=0.05
    ),  # Also show which features generally have the highest error
    aspect=1.75,
)
plot.set_titles("{col_name}")

xticks = sorted(split_index_to_start_time.values())
xticks_before = list(xticks)
xticks = [
    item.split(" ", maxsplit=1)[0] if item.endswith("Q1") else "" for item in xticks
]
plot.set_xticklabels(xticks, rotation=0, horizontalalignment="center")

# from matplotlib.ticker import MultipleLocator, AutoMinorLocator, IndexLocator
# for ax in plot.axes.flat:
#     ax.xaxis.set_minor_locator(IndexLocator(2, 1))
#     ax.xaxis.set_major_locator(IndexLocator(4, 0))

import matplotlib.patches as patches

ymin, ymax = 0.0, 0.6
for ax in plot.axes.flat:
    covid = xticks_before.index("2020 Q1")
    # ax.vlines(
    #     covid + 0.5,
    #     ymin,
    #     ymax,
    #     color="black",
    #     linestyle="--",
    #     label="COVID-19",
    # )
    ax.add_patch(
        patches.Rectangle((covid - 4, ymin), width=8, height=ymax, fill=True, alpha=0.2)
    )

# Label COVID in the upper left plot
first_ax = plot.axes.flat[0]
first_ax.text(
    covid,
    0.45,
    "pandemic",
    ha="center",
    va="center",
    alpha=1.0,
    fontsize="smaller",
)

plot.set(ylim=(ymin, ymax), xlim=(0, len(xticks_before)))
plt.subplots_adjust(hspace=0.35)

sns.move_legend(
    plot,
    "lower right",
    bbox_to_anchor=(0.91, 0.12),
    ncol=3,
    fontsize="medium",
    title_fontsize="larger",
)

plt.savefig(
    OUT_PATH_CROSS / "comparison_error_over_time_per_feature_paper.pdf",
    bbox_inches="tight",
)
plt.show()
plt.close()

In [ ]:
df_all_avg_over_splits = (
    melt_feature(df_cross)
    .groupby(["Models", "metric", "feature"], observed=True)  # not by split
    .mean(numeric_only=True)
    .drop(columns=["split"])
    .reset_index()
)

selected_metric: str = "nCRPS/MAE"
title: str = f"{selected_metric} per model and feature"

df_feat_molten_selected = melt_feature(
    df_all_avg_over_splits[
        df_all_avg_over_splits["metric"] == f"{selected_metric} (avg per feature)"
    ]
)
df_feat_molten_selected["feature"] = (
    df_feat_molten_selected["feature"]
    .apply(FEATURE_NAMES.__getitem__)
    .replace("Cash from Operations", "Cash from  \nOperations")
)

df_feat_molten_selected = df_feat_molten_selected[
    ~df_feat_molten_selected["Models"].isin({"Linear Reg.", "Chronos Base", "Chronos Small"})
]

heatmap_data = (
    df_feat_molten_selected.reset_index()
    .pivot(columns="feature", index="Models", values="value")
    .sort_values(by=["feature"], axis="columns")
)

with sns.axes_style("white"):
    g = sns.JointGrid(
        data=df_feat_molten_selected,
        x="feature",
        y="Models",
        hue="value",
        marginal_ticks=True,
        height=8,
        # ratio=10,
    )

    # Create an inset legend for the histogram colorbar
    # format: (left, bottom, width, height)
    cax = g.figure.add_axes([0.86, 0.83, 0.015, 0.12])

    # Add the joint and marginal histogram plots
    h = sns.heatmap(
        data=heatmap_data,
        ax=g.ax_joint,
        cbar_ax=cax,
        annot=True,
        fmt=".3f",
        annot_kws=dict(fontsize="x-small"),
        xticklabels=True,
        yticklabels=True,
        # cmap="rocket_r",
        cmap="flare",
    )
    cmap = cax.collections[1].get_cmap()
    cmap_norm = cax.collections[1].norm
    h.set_xticklabels(h.get_xticklabels(), rotation=30, horizontalalignment="right")

    barv_data = df_feat_molten_selected.groupby(["feature"])["value"].mean().to_numpy()
    g.ax_marg_x.bar(
        g.ax_joint.get_xticks(),
        barv_data,
        align="center",
        color=cmap(cmap_norm(barv_data)),
    )
    g.ax_marg_x.yaxis.set_ticks_position("left")
    # g.ax_marg_x.yaxis.set_ticks([0, 0.2])

    barh_data = (
        df_feat_molten_selected.groupby(["Models"], observed=True)["value"]
        .mean()
        .to_numpy()
    )
    g.ax_marg_y.barh(
        g.ax_joint.get_yticks(),
        barh_data,
        align="center",
        color=cmap(cmap_norm(barh_data)),
    )
    g.ax_marg_y.xaxis.set_ticks_position("bottom")
    g.ax_marg_y.xaxis.set_ticks([0, 0.2])

    g.ax_joint.set_ylabel("")
    g.ax_joint.set_xlabel("")

    plt.subplots_adjust(top=0.95)
    # plt.suptitle(title)

# plt.tight_layout()
plt.savefig(OUT_PATH_CROSS / "error_per_feature.pdf", bbox_inches="tight")
plt.show()
plt.close()

#### Generate performance comparison tables

In [ ]:
def highlight(data, fun: str = "max"):
    """
    highlight the maximum in a Series or DataFrame + the runner-up
    """
    attr_max = "textbf: --rwrap;"
    attr_runner_up = "underline: --rwrap;"

    # remove % and cast to float
    data = (
        data.str.split("±")
        .apply(lambda x: x[0])
        .apply(lambda x: float("nan") if "nan" in x else x)
        .astype(float)
    )

    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        max_value = getattr(data, fun)()
        is_max = data == max_value
        runner_up_value = getattr(data[data != max_value], fun)()  # can be NaN
        is_runner_up = (
            [False] * len(data)
            if np.isnan(runner_up_value)
            else data == runner_up_value
        )
        return np.where(
            is_max, attr_max, np.where(is_runner_up, attr_runner_up, "")
        ).tolist()
    else:
        max_value = getattr(getattr(data, fun)(), fun)()
        is_max = data == max_value
        runner_up_value = getattr(getattr(data[data != max_value], fun)(), fun)()
        is_runner_up = (
            [False] * len(data.columns)
            if np.isnan(runner_up_value)
            else data == runner_up_value
        )
        return pd.DataFrame(
            np.where(is_max, attr_max, np.where(is_runner_up, attr_runner_up, "")),
            index=data.index,
            columns=data.columns,
        )

In [ ]:
df_single = melt_avg(df_cross)
# df_single = melt_avg_for_human(df_cross)

df_single["metric"] = df_single["metric"].str.split(" ", n=2, expand=True)[0]
df_single.sort_values(by=["metric", "Models"], inplace=True)
df_single = df_single.reset_index()

dfc = (
    df_single.groupby(["Models", "metric"], observed=True)
    .agg({"value": ["mean", "std"]})
    .droplevel(0, axis="columns")
    .reset_index()
)

dfc["value"] = (
    dfc["mean"].apply(lambda x: f"{x:0>2.3f}")
    + "±"
    + dfc["std"].apply(lambda x: f"{x:.2f}")
)
del dfc["mean"]
del dfc["std"]

df = dfc.pivot(index="Models", columns="metric", values="value")

df = df[
    [
        "MAE",
        "MSE",
        "RMSE",
        "MAPE",
        "RSE",
        "sMAPE",
        "R²",
        "nCRPS/MAE",
    ]
]
df.columns = [
    f"{metric} (\\textdownarrow)" if metric != "R²" else metric for metric in df.columns
]
r2_new = "R\\textsuperscript{2} (\\textuparrow)"
df.columns = df.columns.str.replace("R²", r2_new)

df.columns = df.columns.str.replace("nCRPS/MAE", "nCRPS")

styled = df.style.apply(
    highlight, fun="min", subset=list(set(df.columns.tolist()) - {r2_new})
).apply(highlight, fun="max", subset=[r2_new])

print(
    styled.to_latex(
        position_float="centering",
        hrules=True,
        position="th",
        label="tab:comparison_overall",
        caption="Comparison of average performance of the models measured by different metrics. The best is marked as \\textbf{bold} and the runner-up as \\underline{underlined}.",
    )
)
df

### With proper std dev 

In [ ]:
def load_detailed_results(
    base_path: Path, models: set[str] | None = None
) -> pd.DataFrame:
    df_all = pd.read_parquet(base_path / "df_proper.parquet")

    if models is not None:
        df_all = df_all[df_all["model"].isin(models)]
    else:
        df_all = df_all[~df_all["model"].isin(EXCLUDE)]

    return df_all


df_proper = pd.concat(
    [
        load_detailed_results(
            Path("forecast_baselines")
            / "statics_True-revin_True-stride1_lb12-expanding-uq_False",
        ),
        load_detailed_results(
            Path("forecast_baselines")
            / "statics_True-revin_True-stride1_lb12-expanding-uq_True",
        ),
    ]
)

df_proper = df_proper.rename(columns={"model": "Models"})

# df_proper["Models"] = pd.Categorical(df_proper["Models"], order)
df_proper = df_proper.sort_values(by=["Models", "metric"])  # This is slow!

df_proper["metric"] = (
    df_proper["metric"].str.replace("SMAPE", "sMAPE").str.replace("R2", "R²")
)

df_proper

In [ ]:
# sel = df_proper.query("metric == 'RSE' and Models == 'ARMA(4,4)'")
# sel

In [ ]:
dfc = (
    df_proper.groupby(["Models", "metric"], observed=True)
    .agg({"value": ["mean", "std"]})
    .droplevel(0, axis="columns")
    .reset_index()
)

assert not dfc.empty
dfc["value"] = (
    dfc["mean"].apply(lambda x: f"{x:0>2.3f}")
    + "±"
    + dfc["std"].apply(lambda x: f"{x:.2f}")
)
del dfc["mean"]
del dfc["std"]

df = dfc.pivot(index="Models", columns="metric", values="value")

df.loc[df["nCRPS"].isna(), "nCRPS"] = df.loc[df["nCRPS"].isna(), "MAE"]

df = df[
    [
        "MAE",
        "MSE",
        "RMSE",
        "MAPE",
        "RSE",
        "sMAPE",
        "R²",
        "nCRPS",
    ]
]
df.columns = [
    f"{metric} (\\textdownarrow)" if metric != "R²" else metric for metric in df.columns
]
r2_new = "R\\textsuperscript{2} (\\textuparrow)"
df.columns = df.columns.str.replace("R²", r2_new)

df.columns = df.columns.str.replace("nCRPS/MAE", "nCRPS")

styled = df.style.apply(
    highlight, fun="min", subset=list(set(df.columns.tolist()) - {r2_new})
).apply(highlight, fun="max", subset=[r2_new])

print(
    styled.to_latex(
        position_float="centering",
        hrules=True,
        position="th",
        label="tab:comparison_overall",
        caption="Comparison of average performance of the models measured by different metrics. The best is marked as \\textbf{bold} and the runner-up as \\underline{underlined}.",
    )
)
df

### Restricted to human eval 

In [ ]:
human_eval_companies = np.loadtxt("human_eval_companies.csv", delimiter=",")
human_eval_companies.shape

In [ ]:
df_as_human = df_proper
df_as_human = df_as_human[df_as_human["company_id"].isin(human_eval_companies)]
df_as_human = df_as_human.drop(columns=["feature", "horizon_step"])
df_as_human

In [ ]:
# Emulate the averaging over all companies
df_as_human_avg = (
    df_as_human.groupby(["Models", "metric", "split"], observed=True)
    .mean()
    .reset_index()
)
df_as_human_avg

In [ ]:
dfc = (
    df_as_human_avg.groupby(["Models", "metric"], observed=True)
    .agg({"value": ["mean", "std"]})
    .droplevel(0, axis="columns")
    .reset_index()
)

dfc["value"] = (
    dfc["mean"].apply(lambda x: f"{x:0>2.3f}")
    + "±"
    + dfc["std"].apply(lambda x: f"{x:.2f}")
)
del dfc["mean"]
del dfc["std"]

df = dfc.pivot(index="Models", columns="metric", values="value")

df.loc[df["nCRPS"].isna(), "nCRPS"] = df.loc[df["nCRPS"].isna(), "MAE"]

df = df[
    [
        "MAE",
        "MSE",
        "RMSE",
        "MAPE",
        "RSE",
        "sMAPE",
        "R²",
        "nCRPS",
    ]
]
df.columns = [
    f"{metric} (\\textdownarrow)" if metric != "R²" else metric for metric in df.columns
]
r2_new = "R\\textsuperscript{2} (\\textuparrow)"
df.columns = df.columns.str.replace("R²", r2_new)

df.columns = df.columns.str.replace("nCRPS/MAE", "nCRPS")

styled = df.style.apply(
    highlight, fun="min", subset=list(set(df.columns.tolist()) - {r2_new})
).apply(highlight, fun="max", subset=[r2_new])

print(
    styled.to_latex(
        position_float="centering",
        hrules=True,
        position="th",
        label="tab:comparison_overall",
        caption="Comparison of average performance of the models measured by different metrics. The best is marked as \\textbf{bold} and the runner-up as \\underline{underlined}.",
    )
)
df

### Compare with and w/o RevIN

In [ ]:
lb = 12
revin_with = load_results(
    Path("forecast_baselines")
    / f"statics_True-revin_True-stride1_lb{lb}-expanding-uq_True"
)
revin_wo = load_results(
    Path("forecast_baselines")
    / f"statics_True-revin_False-stride1_lb{lb}-expanding-uq_True"
)
revin_with.shape == revin_wo.shape
revin_with

In [ ]:
df = (
    pd.concat(
        {
            "with": melt_avg(revin_with),
            "without": melt_avg(revin_wo),
        },
        names=["RevIN"],
    )
    .reset_index()
    .drop("level_1", axis="columns")
)
df = df.rename(columns={"model": "Models"})
df["Models"] = pd.Categorical(df["Models"], order)
df = df.sort_values(by=["Models", "metric"])
df = df[df["metric"] == "nCRPS (avg)"]
df

In [ ]:
df.loc[df["RevIN"] == "with", "metric"] = "With RevIN"
df.loc[df["RevIN"] == "without", "metric"] = "Without RevIN"

In [ ]:
dfc = (
    df.groupby(["Models", "metric"], observed=True)
    .agg({"value": ["mean", "std"]})
    .droplevel(0, axis="columns")
    .reset_index()
)

dfc["value"] = (
    dfc["mean"].apply(lambda x: f"{x:0>2.3f}")
    + "±"
    + dfc["std"].apply(lambda x: f"{x:.2f}")
)
del dfc["mean"]
del dfc["std"]

df = dfc.pivot(index="Models", columns="metric", values="value")

df.dropna(inplace=True, axis="index")

revin_with = df["With RevIN"].str.split("±", expand=True)[0].astype(float)
revin_wo = df["Without RevIN"].str.split("±", expand=True)[0].astype(float)
df["Improvement"] = (100 * (revin_wo - revin_with) / revin_wo).apply(
    lambda x: f"{x:.2f}\%"
)

df = df[["Without RevIN", "With RevIN", "Improvement"]]  # fix the order

print(
    df.style.to_latex(
        position_float="centering",
        hrules=True,
        position="th",
        label="tab:revin_vs_without",
        caption="A comparison showing the benefit of RevIN.",
    )
)
df

### Compare w/ and w/o statics features

In [ ]:
lb = 12
statics_with = load_results(
    Path("forecast_baselines")
    / f"statics_True-revin_True-stride1_lb{lb}-expanding-uq_True"
)
statics_wo = load_results(
    Path("forecast_baselines")
    / f"statics_False-revin_True-stride1_lb{lb}-expanding-uq_True"
)
statics_with.shape == statics_wo.shape
statics_with

In [ ]:
statics_wo["model"].value_counts()

In [ ]:
len(statics_with["model"].unique()), len(statics_wo["model"].unique())

In [ ]:
df = (
    pd.concat(
        {
            "with": melt_avg(statics_with),
            "without": melt_avg(statics_wo),
        },
        names=["Statics"],
    )
    .reset_index()
    .drop("level_1", axis="columns")
)
df = df.rename(columns={"model": "Models"})
# df["Models"] = pd.Categorical(df["Models"], order)
df = df.sort_values(by=["Models", "metric"])
df = df[df["metric"] == "nCRPS (avg)"]
df

In [ ]:
df["Models"].value_counts()

In [ ]:
# Only compare those models that can actually house static features
df = df[
    df["Models"].isin(
        {
            "Linear Reg.",
            "Random Forest",
            "DLinear",
            "NLinear",
            "TFT",
            "TiDE",
        }
    )
]
df["Models"].value_counts()

In [ ]:
df.loc[df["Statics"] == "with", "metric"] = "With Statics"
df.loc[df["Statics"] == "without", "metric"] = "Without Statics"

In [ ]:
dfc = (
    df.groupby(["Models", "metric"], observed=True)
    .agg({"value": ["mean", "std"]})
    .droplevel(0, axis="columns")
    .reset_index()
)

dfc["value"] = (
    dfc["mean"].apply(lambda x: f"{x:0>2.3f}")
    + "±"
    + dfc["std"].apply(lambda x: f"{x:.2f}")
)
del dfc["mean"]
del dfc["std"]

df = dfc.pivot(index="Models", columns="metric", values="value")

df.dropna(inplace=True, axis="index")

revin_with = df["With Statics"].str.split("±", expand=True)[0].astype(float)
revin_wo = df["Without Statics"].str.split("±", expand=True)[0].astype(float)
df["Improvement"] = (100 * (revin_wo - revin_with) / revin_wo).apply(
    lambda x: f"{x:.2f}\%"
)

df = df[["Without Statics", "With Statics", "Improvement"]]  # fix the order

print(
    df.style.to_latex(
        position_float="centering",
        hrules=True,
        position="th",
        label="tab:benefit_of_statics",
        caption="A comparison showing the benefit of incorporating static features.",
    )
)
df